In [33]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

Empresas Programas por empresa y empresas desde account

In [34]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,Vigencia__c, Corredor__c,Pa_s_del_Paciente__c from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "Vigencia__c",
        "Corredor__c",
        "Pa_s_del_Paciente__c",
    ]
]
Empresas_SF.shape

(6898, 10)

In [35]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6880, 5)

In [36]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id,Name from user
    """
)
usuarios_SF = pd.DataFrame(results["records"])
usuarios_SF = usuarios_SF[
    [
        "Id",
        "Name",
    ]
]
usuarios_SF = usuarios_SF.rename(
    columns={
        "Id": "Id Account Usuario",        
        "Name": "Nombre Usuario",
        
    }
)
usuarios_SF.shape

(80, 2)

In [37]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas.shape

(6904, 15)

In [38]:
Programas_pacientes = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
        "Vigencia__c": "Vigente",
        "Corredor__c": "Corredor",
        "Pa_s_del_Paciente__c": "Pais",
    }
)
Programas_pacientes = Programas_pacientes[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
        "Vigente",
        "Corredor",
        "Pais",
    ]
]

fin empresas y comienzo de rescate de pacientes


In [39]:
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1625675, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [40]:
#buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF.shape

(1188951, 4)

Union de pacientes programas con las empresas

In [41]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_pacientes,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c",
)
empresas_unidas.shape

(1188951, 15)

In [42]:
empresas_unidas = pd.merge(
    left=empresas_unidas,
    right=Pacientes_SF,
    how="left",
    left_on="Paciente__c",
    right_on="Id Account Paciente",
)
empresas_unidas.shape

(1188951, 24)

In [43]:
empresas_unidas = empresas_unidas.rename(columns={"Id": "Id Pacientes Programas"})
empresas_unidas = empresas_unidas[
    [
        "Id Pacientes Programas",
        "Paciente__c",
        "Programas_por_Empresas__c",
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Id Account Paciente",
        "Rut",
        "Nombre",
        "Ocupacion",
        "Cargo",
        "Departamento",
        "Sexo",
        "Telefono",
        "Tipo de Documento",
        "Codigo EMP",
        "Codigo A",
        "Vigente",
        "Corredor",
        "Pais",
    ]
]

empresas_unidas.shape

(1188951, 23)

In [44]:
# cantidad_de_PAcientes = pd.DataFrame()
def cantidad_gente(corredor):
    empresas_unidas2 = empresas_unidas[empresas_unidas["Corredor"] == corredor]
    empresas_unidas2 = empresas_unidas2.drop_duplicates(
        subset=["Rut", "Empleador", "Poliza","Nombre Empresa"], keep="last"
    )
    cantidad_de_PAcientes = empresas_unidas2.groupby(["Poliza", "Empleador"])[
        "Paciente__c"
    ].count()
    cantidad_de_PAcientes = cantidad_de_PAcientes.reset_index()
    # cantidad_de_PAcientes = cantidad_de_PAcientes[cantidad_de_PAcientes["Poliza","Empleador","count"]]
    Archivo_gente = "F:/Reportes SF/MetLife/usuarios/" + corredor + "_gente.xlsx"
    cantidad_de_PAcientes.to_excel(Archivo_gente, index=False)
    cantidad_de_PAcientes.shape

Terminado la visualizacion de las empresas y pacientes que se muestran en los reportes


Descarga desde Citas de Servicios

In [45]:
# Citas de Servicio
results = sf.query_all(
    """
    SELECT Id, clinica__c,AccountId,CreatedDate,RUT__c,Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,Tipo_de_la_cita__c,CreatedById from ServiceAppointment
    """
)


Citas_de_servicio = pd.DataFrame(results["records"])
Citas_de_servicio['Empresas'] = np.nan
#Citas_de_servicio['RUT__c'].map(pacientessf_empresas.set_index('IdentificationId__pc_x',verify_integrity=False)['Name']) 
Citas_de_servicio['Empresas Id'] = np.nan
np.matrix
Citas_de_servicio = Citas_de_servicio[
    [
        "AppointmentNumber",        
        "RUT__c",  
        'Empresas',      
        "CreatedDate",
        "Tipo_de_la_cita__c",
        'Empresas Id',        
        "AccountId",
        'Clinica__c',
        'CreatedById'
        
    ]
]
Citas_de_servicio["AppointmentNumber"] = 'F' + Citas_de_servicio["AppointmentNumber"]
    
Citas_de_servicio.head()


,AppointmentNumber,RUT__c,Empresas,CreatedDate,Tipo_de_la_cita__c,Empresas Id,AccountId,Clinica__c,CreatedById
0,FSA-12257,138959341,NaN,2023-03-11T12:30:20.000+0000,Video,NaN,0018c00002amSUbAAM,Clínica de Psicología,0058c00000DcdOjAAJ
1,FSA-3124,183552252,NaN,2022-12-02T16:23:05.000+0000,Phone,NaN,0018c00002azpJ4AAI,Clínica de Nutrición,0058c00000DcdNnAAJ
2,FSA-11374,139187598,NaN,2023-03-03T17:16:41.000+0000,Phone,NaN,0018c00002amLVAAA2,Clínica de Nutrición,0058c00000DcdO5AAJ
3,FSA-10627,159754278,NaN,2023-02-24T13:00:37.000+0000,Phone,NaN,0018c00002bINLlAAO,Clínica de Nutrición,0058c00000DcdNnAAJ
4,FSA-12261,162332392,NaN,2023-03-11T12:44:57.000+0000,Phone,NaN,0018c00002cX820AAC,Clínica APSSex,0058c00000DcdOAAAZ


In [46]:
Citas_de_servicio.to_excel("F:/Empresas SF/reportes/citas de servicio.xlsx", index=False)

In [47]:
# Citas de Servicio Orientacion
results = sf.query_all(
    """
    SELECT Id, AccountId,CreatedDate,RUT__c,WorkTypeId,Work_TypeGroup__c,Status,Interacci_n__c,Tipo_de_la_cita__c,AppointmentNumber,CreatedById from ServiceAppointment
    """
)
Citas_de_servicio2 = pd.DataFrame(results["records"])
Citas_de_servicio2['Clinica__c'] = 'Orientación Médica'
Citas_de_servicio2['Empresas'] = np.nan
Citas_de_servicio2['Empresas Id'] = np.nan
Citas_de_servicio2 = Citas_de_servicio2[
    [
        "AppointmentNumber",        
        "RUT__c",  
        'Empresas',      
        "CreatedDate",
        "Tipo_de_la_cita__c",
        'Empresas Id',       
        "AccountId",
        'Clinica__c',
        'CreatedById'
        
    ]
]
Citas_de_servicio2.shape


(14695, 9)

In [48]:
#unir dos citas
Gran_Citas_deservicio = pd.concat([Citas_de_servicio, Citas_de_servicio2], axis=0)
Gran_Citas_deservicio = Gran_Citas_deservicio.rename(
    columns={
        "AppointmentNumber": "Codigo Ficha",
        "RUT__c": "Rut",
        "Empresas": "Empleador",
        "CreatedDate": "Fecha",
        "Tipo_de_la_cita__c": "Canal",
        "Empresas Id": "Cod. Empresa",
        "AccountId": "Id Paciente",        
        'Clinica__c':'Clínica',
        'CreatedById':'Creado'
    }
)

Gran_Citas_deservicio.shape

(29390, 9)

In [49]:
#Clinica_deporte
results = sf.query_all(
    """
   SELECT id,Name,Rut__c,Nombre_Empresa__c,CreatedDate,Canal__c, Empresa__c, Account__c,CreatedById from Ficha_Cl_nica_Deporte__c
     """
)
ficha_deporte = pd.DataFrame(results["records"])

ficha_deporte = ficha_deporte[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'CreatedById',
        
    ]
]
ficha_deporte['clinica'] = 'Deporte'
ficha_deporte = ficha_deporte.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'clinica':'Clínica',
        'CreatedById':'Creado'
    }
            )
ficha_deporte.shape


(2041, 9)

In [50]:
# Clinica Nutricion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c,CreatedById from Ficha_Clinica_Nutrici_n__c 
     """
)
ficha_nutricion = pd.DataFrame(results["records"])

ficha_nutricion = ficha_nutricion[
    [
        "Name",
        "RUT__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'CreatedById',
        
    ]
]
ficha_nutricion["clinica"] = "Nutrición"

ficha_nutricion = ficha_nutricion.rename(
    columns={
        "Name": "Codigo Ficha",
        "RUT__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica',
        'CreatedById':'Creado'
    }
)


ficha_nutricion.shape

(4508, 9)

In [51]:
#Clinica AppSex
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId,CreatedById from Ficha_Cl_nica_ApsSex__c 
     """
)
ficha_Apssex = pd.DataFrame(results["records"])

ficha_Apssex = ficha_Apssex[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'CreatedById',
        
    ]
]
ficha_Apssex['clinica'] = 'Asesoría Psicologíca en Salud Sexual'
ficha_Apssex = ficha_Apssex.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica',
        'CreatedById':'Creado'
    }
)

ficha_Apssex.shape

(19, 9)

In [52]:
# Clinica Psicologia
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId,Motivo_de_consulta_manifiesto__c,CreatedById from Ficha_Cl_nica_Asistencia_Emocional__c 
     """
)
ficha_Psicologia = pd.DataFrame(results["records"])

ficha_Psicologia = ficha_Psicologia[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "Motivo_de_consulta_manifiesto__c",
        'CreatedById',
    ]
]
ficha_Psicologia["clinica"] = "Psicología"
ficha_Psicologia.Motivo_de_consulta_manifiesto__c.fillna(0, inplace=True)
ficha_Psicologia['Motivo_de_consulta_manifiesto__c'] = np.where(ficha_Psicologia['Motivo_de_consulta_manifiesto__c']== 0 ,'Otras Derivaciones',ficha_Psicologia['Motivo_de_consulta_manifiesto__c'] )
#ficha_Psicologia['77'] = len(ficha_Psicologia['Motivo_de_consulta_manifiesto__c'])
ficha_Psicologia = ficha_Psicologia.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        "Motivo_de_consulta_manifiesto__c": "Motivo de consulta",
        'CreatedById':'Creado'
    }
)

ficha_Psicologia.shape

(3133, 10)

In [53]:
#Clinica Sueño
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId,CreatedById from Ficha_Cl_nica_del_Sue_o__c 
     """
)
ficha_Sueno = pd.DataFrame(results["records"])
ficha_Sueno = ficha_Sueno[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'CreatedById',
    ]
]
ficha_Sueno['clinica'] = 'Clínica Del Sueño'
ficha_Sueno = ficha_Sueno.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica',
        'CreatedById':'Creado'
    }
)
ficha_Sueno.shape

(416, 9)

In [54]:
#Clinica PAciente Cronico
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c,CreatedById from Ficha_Cl_nica_Seg_Paciente_Cr_nico__c  
     """
)
ficha_Cronico = pd.DataFrame(results["records"])
ficha_Cronico = ficha_Cronico[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'CreatedById'
    ]
]
ficha_Cronico['clinica'] = 'Pacientes Crónicos'
ficha_Cronico = ficha_Cronico.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica',
        'CreatedById':'Creado'
    }
)
ficha_Cronico.shape

(1598, 9)

In [55]:
#Clinica Telemedicina
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c,CreatedById from Ficha_Cl_nica_Telemedicina__c   
     """
)
ficha_Telemedicina = pd.DataFrame(results["records"])
ficha_Telemedicina = ficha_Telemedicina[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'CreatedById'
    ]
]
ficha_Telemedicina['clinica'] = 'Telemedicina'
ficha_Telemedicina = ficha_Telemedicina.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'clinica':'Clínica',
        'CreatedById':'Creado'
    }
)
ficha_Telemedicina.shape

(5151, 9)

In [56]:
#Clinica Veterinaria
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c,CreatedById, Account__c from Ficha_Cl_nica_Veterinaria__c    
     """
)
ficha_Veterinaria = pd.DataFrame(results["records"])
ficha_Veterinaria = ficha_Veterinaria[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'CreatedById'
    ]
]
ficha_Veterinaria['clinica'] = 'Veterinaria'
ficha_Veterinaria = ficha_Veterinaria.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica',
        'CreatedById':'Creado'
    }
)
ficha_Veterinaria.shape

(45, 9)

In [57]:
# Clinica Orientacion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c,CreatedById from Ficha_Orientaci_n_Medica__c     
     """
)
ficha_Orientacion = pd.DataFrame(results["records"])
ficha_Orientacion = ficha_Orientacion[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'CreatedById'
    ]
]
ficha_Orientacion["clinica"] = "Orientación Médica"
ficha_Orientacion = ficha_Orientacion.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        'CreatedById':'Creado'
    }
)
ficha_Orientacion.shape

(109, 9)

In [58]:
# Clinica Front
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c,CreatedById from Ficha_Front__c     
     """
)
ficha_Front = pd.DataFrame(results["records"])

ficha_Front = ficha_Front[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'CreatedById'
    ]
]
ficha_Front["clinica"] = "Orientación Médica"
ficha_Front = ficha_Front.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        'CreatedById':'Creado'
    }
)
ficha_Front.shape

(1149, 9)

In [59]:
#Clinica_general
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c,Account__c, Programa__c,CreatedById   from Ficha_Clinica_General__c 
     """
)
ficha_general = pd.DataFrame(results["records"])

ficha_general = ficha_general[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'Programa__c',
        'CreatedById'
        
    ]
]

ficha_general = ficha_general.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'Programa__c':'Clínica',
        'CreatedById':'Creado'
    }
)
ficha_general.shape


(1992, 9)

In [60]:
#Clinica Ingreso deportivo
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c,CreatedById from Ficha_Cl_nica_Ingreso_Deportivo__c    
     """
)
ficha_Ingreso_dep = pd.DataFrame(results["records"])

ficha_Ingreso_dep = ficha_Ingreso_dep[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'CreatedById'
        
    ]
]
ficha_Ingreso_dep['clinica'] = 'Ingreso Deporte Seguro'
ficha_Ingreso_dep = ficha_Ingreso_dep.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica',
        'CreatedById':'Creado'
    }
)
ficha_Ingreso_dep.shape

(268, 9)

In [61]:
#Clinica mis primeros pasos
results = sf.query_all(
    """
   select  Name,Rut__c,Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Cuenta__c,  CreatedById from Ficha_Mis_Primeros_Pasos__c   
     """
)
ficha_Mis_Pri = pd.DataFrame(results["records"])

ficha_Mis_Pri = ficha_Mis_Pri[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Cuenta__c",
        'CreatedById'
        
    ]
]
ficha_Mis_Pri['clinica'] = 'Mis Primeros Pasos'
ficha_Mis_Pri = ficha_Mis_Pri.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Cuenta__c": "Id Paciente",
        'clinica':'Clínica',
        'CreatedById':'Creado'
    }
)
ficha_Mis_Pri.shape

(5, 9)

In [62]:
#Clinica Ergonomia
results = sf.query_all(
    """
   select  Name,Rut__c,Nombre_de_Empresa__c,  CreatedDate, Canal__c, Empresa__c, Cuenta__c,  CreatedById from Ficha_de_Ergonomia__c    
     """
)
Ficha_Ergonomia = pd.DataFrame(results["records"])

Ficha_Ergonomia = Ficha_Ergonomia[
    [
        "Name",
        "Rut__c",
        "Nombre_de_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Cuenta__c",
        'CreatedById'
        
    ]
]
Ficha_Ergonomia['clinica'] = 'Ergonomía'
Ficha_Ergonomia = Ficha_Ergonomia.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_de_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Cuenta__c": "Id Paciente",
        'clinica':'Clínica',
        'CreatedById':'Creado'
    }
)
Ficha_Ergonomia.shape

(1, 9)

In [63]:
# Ficha Activacion
results = sf.query_all(
    """
   select  Name,Rut__c, Empresas__c, Nombre_Empresa__c,  CreatedDate, Area__c,  Cuenta__c,  CreatedById from Activaciones__c    
     """
)
Ficha_Activacion = pd.DataFrame(results["records"])

Ficha_Activacion = Ficha_Activacion[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Empresas__c",
        "Cuenta__c",
        "CreatedById",
        "Area__c",
    ]
]
Ficha_Activacion["Canal"] = "Presencial"
Ficha_Activacion = Ficha_Activacion.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_de_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Empresa__c": "Cod. Empresa",
        "Cuenta__c": "Id Paciente",
        "clinica": "Clínica",
        "CreatedById": "Creado",
        "Area__c": "Clínica",
    }
)
Ficha_Activacion.shape

(304, 9)

In [64]:
Clinicas = pd.concat(
    [
        ficha_Front,
        ficha_Ingreso_dep,
        ficha_Orientacion,
        ficha_Veterinaria,
        ficha_Telemedicina,
        ficha_Cronico,
        ficha_Sueno,
        ficha_Psicologia,
        ficha_Apssex,
        ficha_nutricion,
        ficha_deporte,
        ficha_general,
        ficha_Mis_Pri,
        Ficha_Ergonomia,
        Ficha_Activacion,
    ],
    axis=0,
)
Clinicas.shape

(20739, 12)

In [65]:
Reortes_general = pd.concat([Clinicas,Gran_Citas_deservicio ], axis=0)
Reortes_general.shape

(50129, 12)

In [66]:
Reortes_general["Dia"] = pd.to_datetime(Reortes_general["Fecha"]).dt.day

Reortes_general["Mes"] = pd.to_datetime(Reortes_general["Fecha"]).dt.month

Reortes_general["semana"] = pd.to_datetime(Reortes_general["Fecha"]).dt.month_name()

Reortes_general["Año"] = pd.to_datetime(Reortes_general["Fecha"]).dt.year


# x = date(Citas_de_servicio['Dia'], Citas_de_servicio['Mes'], Citas_de_servicio['Año'] )
Reortes_general["Fecha_chilena"] = pd.to_datetime(
    Reortes_general["Fecha"], dayfirst=True, yearfirst=False, exact=True
)
Reortes_general["Fecha_chilena2"] = Reortes_general["Fecha_chilena"].dt.strftime(
    "%d-%m-%Y"
)
general_Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        'Varón':'Hombre'      
        
    }


clinicas_ = {
    "Clínica de Nutrición": "Nutrición",
    "Clínica de Deporte": "Deporte",
    "Clínica de Psicología": "Psicología",
    "Clínica Ingreso Deportivo Seguro": "Deporte",
    "Clínica del Sueño": "Clínica Del Sueño",
    "Clínica Seguimiento Pacientes Crónicos": "Pacientes Crónicos",
    "Orientación Médica": "Orientación Médica",
    "Asesoría Deportiva": "Deporte",
    "Asistencia Emocional": "Psicología",
    "Asistencia Nutricional": "Nutrición",
    "Clínica APSSex": "Asesoría Psicologíca en Salud Sexual",
    "Clínica Del Sueño": "Clínica Del Sueño",
    "Clínica Veterinaria": "Veterinaria",
    "Ingreso Deportivo Seguro": "Deporte",
    "Clínica Mis Primeros Pasos": "Mis Primeros Pasos",
    "Ingreso Deporte Seguro": "Deporte",
}
Reortes_general.replace({"Clínica": clinicas_}, inplace=True)

Clinicas_metlife = {
    "Mis Primeros Pasos": "Pacientes Crónicos",
    "Ingreso Deporte Seguro": "Deporte",
    "Veterinaria": "Orientación Médica",
}

Canal = {
    "Phone": "Teléfono",
    "Video": "Videollamada",
    "Entrante - Chat": "Entrante - Chat Online",
}
Reortes_general.replace({"Canal": Canal}, inplace=True)

meses = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
    }
Reortes_general['Meses'] = Reortes_general['Mes']
Reortes_general.replace({"Meses": meses}, inplace=True)
Reortes_general["Fecha_chilena"] = Reortes_general["Fecha_chilena"].dt.tz_localize(None)
Reortes_general.Rut.fillna("0", inplace=True)
Reortes_general.Canal.fillna("Sin Canal", inplace=True)


In [75]:
Reortes_general.head()

,Codigo Ficha,Rut,Empleador,Fecha,Canal,Cod. Empresa,Id Paciente,Creado,Clínica,Motivo de consulta,Nombre_Empresa__c,Empresas__c,Dia,Mes,semana,Año,Fecha_chilena,Fecha_chilena2,Meses
0,FFF-0358,200166892,Farmacias Cruz Verde S.A.,2023-02-13T18:31:03.000+0000,Entrante - Chat Online,a1Y8c00000C6VqyEAF,0018c00002amLv7AAE,0058c00000DcdNVAAZ,Orientación Médica,NaN,NaN,NaN,13,2,February,2023,2023-02-13 18:31:03,13-02-2023,Febrero
1,FFF-0035,181523778,Farmacias Ahumada S.A.,2023-01-24T18:31:33.000+0000,Entrante - WhatsApp,a1Y8c00000C6VINEA3,0018c00002amVPcAAM,0058c00000DcdNYAAZ,Orientación Médica,NaN,NaN,NaN,24,1,January,2023,2023-01-24 18:31:33,24-01-2023,Enero
2,FFF-0154,166090636,EY Audit SPA,2023-01-30T18:24:32.000+0000,Entrante - Chat Online,a1Y8c00000DxPM9EAN,0018c00002amSFbAAM,0058c00000DcdNVAAZ,Orientación Médica,NaN,NaN,NaN,30,1,January,2023,2023-01-30 18:24:32,30-01-2023,Enero
3,FFF-0088,164841448,Evaluaserve Chile S.A.,2023-01-26T14:25:58.000+0000,Entrante - Chat Online,a1Y8c00000Clq5uEAB,0018c00002b03a0AAA,0058c00000DcdNWAAZ,Orientación Médica,NaN,NaN,NaN,26,1,January,2023,2023-01-26 14:25:58,26-01-2023,Enero
4,FFF-0096,188075606,None,2023-01-26T17:29:57.000+0000,Entrante - Llamada,None,0018c00002eimgBAAQ,0058c00000DcdNVAAZ,Orientación Médica,NaN,NaN,NaN,26,1,January,2023,2023-01-26 17:29:57,26-01-2023,Enero


In [76]:
empresas_unidas.head()

,Id Pacientes Programas,Paciente__c,Programas_por_Empresas__c,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Id Account Paciente,...,Cargo,Departamento,Sexo,Telefono,Tipo de Documento,Codigo EMP,Codigo A,Vigente,Corredor,Pais
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,0018c00002amLbpAAE,...,None,None,Mujer,967786319,RUT,EMP6008,A-0173,SI,Directo BCH,a1U8c000005dAtnEAE
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLbqAAE,...,None,None,Mujer,None,RUT,EMP6274,A-0353,SI,Willis Towers Watson S.A. Corred. De Seg.,a1U8c000005dAtnEAE
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLbrAAE,...,None,None,Mujer,None,RUT,EMP6274,A-0353,SI,Willis Towers Watson S.A. Corred. De Seg.,a1U8c000005dAtnEAE
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLbsAAE,...,None,None,Hombre,953757757,RUT,EMP6274,A-0353,SI,Willis Towers Watson S.A. Corred. De Seg.,a1U8c000005dAtnEAE
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLc2AAE,...,None,None,Mujer,982058845,RUT,EMP6274,A-0353,SI,Willis Towers Watson S.A. Corred. De Seg.,a1U8c000005dAtnEAE


In [67]:
Reportes_con_empresas = pd.merge(
    left=Reortes_general,
    right=empresas_unidas,
    how="left",
    left_on="Id Paciente",
    right_on="Id Account Paciente",
)


Reportes_con_empresas.shape

(59536, 42)

In [68]:
Reportes_con_empresas['Rut_x'].str.upper()
Reportes_con_empresas['Rut_y'].str.upper()

Reportes_con_empresas.Empleador_x.fillna(Reportes_con_empresas['Nombre Empresa'], inplace=True)

In [70]:
Programas_pacientes.to_excel("C:/Users/jmartinez/Documents/Empresas.xlsx", index=False)

Salida de datos

In [71]:
reportes = Reportes_con_empresas[
    [
        "Codigo Ficha",
        "Rut_x",
        "Nombre",
        "Sexo",
        "Ocupacion",
        "Cargo",
        "Departamento",
        "Campaña",
        "Holding",
        "Empleador_x",
        "Poliza",
        "Fecha",
        "Fecha_chilena",
        "Fecha_chilena2",
        "Dia",
        "Mes",
        "Año",
        "Meses",
        "Canal",
        "Clínica",
        "Motivo de consulta",
        "Nombre Empresa",
        "Codigo EMP",
        "Codigo A",
        "Vigente",
        "Corredor",
        "Pais",
    ]
]

Pais_SF = {"a1U8c000005dAtnEAE": "Chile", "a1U8c000005dAtsEAE": "México"}
reportes.replace({"Pais": Pais_SF}, inplace=True)

reportes = reportes.rename(columns={"Rut_x": "Rut", "Empleador_x": "Empleador"})
reportes["Motivo de consulta"].fillna(0, inplace=True)
reportes["Motivo de consulta"] = np.where(
    (reportes["Clínica"] == "Psicología") & (reportes["Motivo de consulta"] == 0),
    "Ingreso a Psicologia",
    reportes["Motivo de consulta"],
)
reportes.Clínica.fillna("Orientación Médica", inplace=True)
reportes = reportes.drop_duplicates(
    ["Clínica", "Empleador", "Nombre Empresa", "Codigo Ficha"], keep="last"
)

reportes['Rut'] = reportes['Rut'].str.replace("-","")
reportes.replace({"Sexo": general_Sexo}, inplace=True)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15020\121612437.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reportes.replace({"Pais": Pais_SF}, inplace=True)


In [73]:
Reportes_con_empresas.to_excel(
    "C:/Users/jmartinez/Documents/reportes_completos.xlsx", index=False
)


Programas_pacientes.to_excel("C:/Users/jmartinez/Documents/Empresas.xlsx", index=False)
reportes.to_excel("C:/Users/jmartinez/Documents/reportes.xlsx", index=False)

In [74]:
general_todo = reportes[(reportes["Empleador"] == reportes["Nombre Empresa"])]
general_todo.to_excel("F:/Reportes SF/General/General_Base.xlsx", index=False)